In [51]:
#bechmark for the querry
import duckdb
import mongo_deps as mongo
import duckdb_deps as duck
import time
import numpy as np
import pandas as pd
import os

# create the structure of the benchmark results folder
if not os.path.exists("./benchmark_results"):
    os.makedirs("./benchmark_results")
if not os.path.exists("./benchmark_results/mongoDB"):
    os.makedirs("./benchmark_results/mongoDB")
if not os.path.exists("./benchmark_results/duckDB"):
    os.makedirs("./benchmark_results/duckDB")

def run(querry,benchmark_duckdb,connection = None,cursor = None):

    if benchmark_duckdb:
        result = cursor.execute(querry)
    else:
        result = querry.run_query()
    return result
        

def benchmark_round(querry,benchmark_duckdb,connection,cursor):
    startTime = time.time()
    non_materilised = run(querry,benchmark_duckdb,connection,cursor)
    endTime = time.time()
    if benchmark_duckdb:
        materialize = non_materilised.fetchdf()
    
    materializeTime =  time.time()-endTime 
    querryTime = endTime - startTime

    return querryTime,materializeTime

def benchmark(querry, rounds,benchmark_duckdb,querryName,connection,cursor):
    times = []
    for i in range(rounds):
        times.append(benchmark_round(querry,benchmark_duckdb,connection,cursor))
    
    #save the benchmark results to the appropriate folder
    savingFolder = ["mongoDB","duckDB"][benchmark_duckdb]

    pd.DataFrame(times).to_csv(f"./benchmark_results/{savingFolder}/{querryName}_full.csv")
    return np.array(times)




# create a summury of the benchmark
def benchmark_summary_per_querry(times:np.array):
    return {"average_querry_time": times[:,0].mean(), "max_querry_time": times[:,0].max(), "min_querry_time": times[:,0].min(), "standard_deviation_querry_time": times[:,0].std(), "average_materialize_time": times[:,1].mean(), "max_materialize_time": times[:,1].max(), "min_materialize_time": times[:,1].min(), "standard_deviation_materialize_time": times[:,1].std()}


In [3]:
#create a pandas dataframe with the benchmark results
def benchmark_summary(querrys, rounds,benchmark_duckdb,querryNames,connection,cursor):
    import pandas as pd
    savingFolder = ["mongoDB","duckDB"][benchmark_duckdb]
    result = {}
    for index in range(len(querrys)):
        #(querry, rounds,benchmark_duckdb,querryName,connection,cursor
        times = benchmark(querrys[index], rounds,benchmark_duckdb,querryNames[index],connection,cursor)
        result[querryNames[index]] = benchmark_summary_per_querry(times)

    result = pd.DataFrame(result)
    result.columns = querryNames
    return result.T

# MongoDB

In [4]:
# bemchaark mongodb querries 
finalMongoDB = benchmark_summary(mongo.get_mongo_queries(),20,benchmark_duckdb=False,querryNames=mongo.get_mongo_queries_names(),connection=None,cursor=None)
finalMongoDB.to_csv(f"./benchmark_results/mongoDB/mongoDB_summary.csv")
finalMongoDB

,average,max,min,standard_deviation
MongoDB_E1,0.011935,0.050581,0.008877,0.008907
MongoDB_E2,2.027207,4.131138,1.869610,0.483585
MongoDB_M1,1.840971,3.760936,1.671002,0.446469
MongoDB_M2,5.415158,6.089313,5.239760,0.186563
MongoDB_M3,1.240256,1.325222,1.215517,0.027894
MongoDB_M4,1.825203,1.882356,1.789892,0.022035
MongoDB_M5,0.338390,0.557600,0.275877,0.072637
MongoDB_M6,14.574113,17.161940,13.675758,0.710377


# DuckDB

In [5]:
#initialize duckdb
conn,cursor = duck.init_duckdb()

In [52]:
# bemchaark duckdb querries

finalDuckDB = benchmark_summary(duck.get_duckdb_queries(),20,benchmark_duckdb=True,querryNames=duck.get_duckdb_queries_names(),connection=conn,cursor=cursor)
finalDuckDB.to_csv(f"./benchmark_results/duckDB/duckDB_summary.csv")
finalDuckDB

,average_querry_time,max_querry_time,min_querry_time,standard_deviation_querry_time,average_materialize_time,max_materialize_time,min_materialize_time,standard_deviation_materialize_time
E1,0.008894,0.011001,0.006966,0.001234,0.001309,0.003054,0.000972,0.000565
E2,0.064576,0.073002,0.059999,0.003941,0.000787,0.001011,0.000000,0.000398
M1,0.058627,0.069001,0.054000,0.003392,0.000885,0.001011,0.000000,0.000302
M2,0.013544,0.017001,0.011871,0.001292,0.001906,0.004004,0.000999,0.000702
M3,0.043772,0.053261,0.037998,0.004254,0.000691,0.001002,0.000000,0.000454
M4,0.042956,0.045225,0.040999,0.001368,0.000956,0.001119,0.000000,0.000221
M5,0.103816,0.115975,0.022001,0.019181,1.888774,2.085228,1.832583,0.053500
M6,6.882647,7.246406,6.641041,0.166505,0.500103,0.639349,0.446051,0.048010
